In [41]:
# assume @ is the epsilon symbol
class edge:
    def __init__(self,src,dist,label="@"):
        self.src = src
        self.dist = dist
        self.label = label

In [42]:
sid=0

class state:
    def __init__(self,edges):
        global sid
        self.Outedges:list[edge] = edges
        self.label = "S"+str(sid)
        sid+=1
    def add_edge(self,edge):
        self.Outedges.append(edge)

In [43]:
from graphviz import Digraph

class NFA:
    def __init__(self,start,accept,inner_states):
        self.start:state = start
        self.accept:state = accept
        self.inner_states:list[state] =inner_states

    def display(self):
        print("startingState:",self.start.label,",")
        for s in self.inner_states:
            print(s.label,": {")
            print(" isTerminatingState : ",s.label==self.accept.label,",    ")
            for e in s.Outedges:
                print(e.label,":",e.dist.label,",")
            print("},\n")

    def graph (self):
        dot = Digraph(comment='NFA graph',format='png',graph_attr={ 'rankdir': 'LR'})
        # dot.node(self.start.label,self.start.label,shape="circle")
        # dot.node(self.accept.label,self.accept.label,shape="doublecircle")
        for s in self.inner_states:
            if s.label==self.accept.label:
                dot.node(s.label,s.label,shape="doublecircle")
            else:
                dot.node(s.label,s.label,shape="circle")
            for e in s.Outedges:
                dot.edge(e.src.label,e.dist.label,e.label)
        dot.render('test-output/round-table.gv', view=True)
        


In [44]:
def Concat(stack:list[NFA]):
    nfa2:NFA = stack.pop()
    nfa1:NFA = stack.pop()
    
    newEdges = edge(nfa1.accept,nfa2.start)
    nfa1.accept.add_edge(newEdges)

    resNfa = NFA(nfa1.start,nfa2.accept,nfa1.inner_states+nfa2.inner_states)
    stack.append(resNfa)

    return stack

In [45]:
def Or(stack:list[NFA]):
    nfa1 = stack.pop()
    nfa2 = stack.pop()
    
    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1
    
    newEdges1 = edge(newStart,nfa1.start)
    newEdges2 = edge(newStart,nfa2.start)

    newStart.add_edge(newEdges1)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa1.accept,newEnd)
    newEdges4 = edge(nfa2.accept,newEnd)
    
    nfa1.accept.add_edge(newEdges3)
    nfa2.accept.add_edge(newEdges4)

    resNfa = NFA(newStart,newEnd,nfa1.inner_states+nfa2.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [46]:
def ZeroMore(stack:list[NFA]):
    nfa = stack.pop()
    
    newEdges1 = edge(nfa.accept,nfa.start)
    nfa.accept.add_edge(newEdges1)

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges2 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges3)

    newEdges4 = edge(newStart,newEnd)
    newStart.add_edge(newEdges4)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [47]:
def OneMore(stack:list[NFA]):
    nfa = stack.pop()
    
    newEdges1 = edge(nfa.accept,nfa.start)
    nfa.accept.add_edge(newEdges1)

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges2 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges2)

    newEdges3 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges3)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [48]:
def ZeroOne(stack:list[NFA]):
    nfa = stack.pop()

    newStart = state([])
    # sid+=1
    newEnd = state([])
    # sid+=1

    newEdges1 = edge(newStart,nfa.start)
    newStart.add_edge(newEdges1)

    newEdges2 = edge(nfa.accept,newEnd)
    nfa.accept.add_edge(newEdges2)

    newEdges3 = edge(newStart,newEnd)
    newStart.add_edge(newEdges3)

    resNfa = NFA(newStart,newEnd,nfa.inner_states+[newStart,newEnd])
    stack.append(resNfa)
    
    return stack

In [49]:
def Construct(stack:list[NFA],s:str):
    
    start=state([])
    # sid+=1
    accept=state([])
    # sid+=1
    
    newEdges = edge(start,accept,s)
    start.add_edge(newEdges)

    resNfa = NFA(start,accept,[start,accept])
    stack.append(resNfa)

    return stack

In [50]:
def PostfixToNFA(postfix:str):
    alphanum = {char:char for char in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789."}
    stack = []
    for c in postfix:
        if c in alphanum:
            stack = Construct(stack,c)
        elif c == "*":
            stack = ZeroMore(stack)
        elif c == "+":
            stack = OneMore(stack)
        elif c == "?":
            stack = ZeroOne(stack)
        elif c == ".":
            stack = Concat(stack)
        elif c == "|":
            stack = Or(stack)
        else:
            print("Invalid postfix expression character {c} is not recognized")
            return None
    if len(stack)!=1:
        print("Invalid postfix expression")
        return None
    
    result = stack.pop()
        
    return result

In [51]:
# test cell 
# expression = "(A+B*)?(C|D)"
# postfix = "A+B*.?CD|."
postfix="A+B*.?CD|."
nfa = PostfixToNFA(postfix)
nfa.display()
nfa.graph()


startingState: S0 ,
S0 : {
 isTerminatingState :  False ,    
a : S1 ,
},

S1 : {
 isTerminatingState :  False ,    
@ : S8 ,
},

S2 : {
 isTerminatingState :  False ,    
a : S3 ,
},

S3 : {
 isTerminatingState :  False ,    
@ : S2 ,
@ : S5 ,
},

S4 : {
 isTerminatingState :  False ,    
@ : S2 ,
},

S5 : {
 isTerminatingState :  False ,    
@ : S6 ,
},

S6 : {
 isTerminatingState :  False ,    
b : S7 ,
},

S7 : {
 isTerminatingState :  False ,    
@ : S4 ,
@ : S9 ,
},

S8 : {
 isTerminatingState :  False ,    
@ : S4 ,
@ : S9 ,
},

S9 : {
 isTerminatingState :  False ,    
@ : S10 ,
},

S10 : {
 isTerminatingState :  False ,    
b : S11 ,
},

S11 : {
 isTerminatingState :  True ,    
},



ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH